In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json
from skimage.feature import hog
import cv2

print(tf.__version__)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


1.10.0


In [2]:
#Helper library
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Returns a short sequential model
def create_model():
    model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [4]:
# load json and create model
#json_file = open('model.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#model = model_from_json(loaded_model_json)
model = create_model()
model = model.load_weights('digit.h5')

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

AttributeError: 'NoneType' object has no attribute 'compile'

In [ ]:
im = cv2.imread('photo_1.jpg')
# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
       
# Threshold the image
ret, im_th = cv2.threshold(im_gray, 120, 255, cv2.THRESH_BINARY_INV)

# Find contours in the image
ima, ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(im, ctrs, -1, (173,213,0), 3)
    
if len(ctrs) > 0:
    for contour in ctrs:
        if (cv2.contourArea(contour) > 300):
            # Get rectangles contains each contour
            rect = cv2.boundingRect(contour) 
            # Draw the rectangles
            cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
            # Make the rectangular region around the digit
            leng = int(rect[3] * 1.6)
            pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
            pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
            roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
            # Resize the image
            roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
            roi = cv2.dilate(roi, (3, 3))
            roi = np.array(roi)
            roi = roi.astype('float32')
            roi /= 255

            roi = np.expand_dims(roi, axis=0)
            ans = model.predict(roi).argmax()

            print('answer: ',ans)